In [ ]:
# 강의실 모델 최종 
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from google.colab import drive
drive.mount('/content/drive')



# 강의실 데이터
class_data = pd.read_csv('/content/drive/MyDrive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/강의실/강의실 건물별 TOP3 최종.csv', sep = ",",error_bad_lines=False) 
class_data.insert(4,'plugs_per_stu', class_data['plugs'] / class_data['max'])
class_data.insert(4, 'plug_necessity', pd.cut(class_data.plugs_per_stu, 5))
class_data['plug_necessity'] = class_data['plug_necessity'].astype('str')
class_data = class_data.replace(['(-0.00395, 0.79]','(0.79, 1.58]', '(1.58, 2.37]', '(2.37, 3.16]','(3.16, 3.95]'],[1,2,3,4,5]) # 콘센트 필요도 변환
class_data_1 = class_data.drop(columns=["classroom_index" ,"max", "plugs_per_stu","plugs"]) # 호수 column 유지(코사인 유사도 계산 시에만 배제)
class_data_1= class_data_1.fillna(0)


# 건물 간 이동시간 데이터
distance = pd.read_csv('/content/drive/MyDrive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/건물별 이동시간 최종.csv', sep = ",",error_bad_lines=False) 



# 코사인유사도 계산 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))



# max min 정규화 함수
def minmax_norm(df_input):
    return df_input.apply(lambda x: (x-x.min())/ (x.max() - x.min()))




# User Input에 대한 이동시간 col 생성 함수
def make_distance(user):
  ls = [] # class_data_1의 전체 행에 대한 이동시간 리스트
  temp = distance[distance.building == user[1]]
  temp = temp.reset_index(drop=True)
  
  for idx, row in class_data_1.iterrows():
    distance_user = temp.loc[0][class_data_1.loc[idx]['building']]
    ls.append([class_data_1.loc[idx]['building'], distance_user])

  temp2 = pd.DataFrame(ls, columns= [ 'building' ,'distance_time'])
  temp2['distance_time'] = (temp2['distance_time'] - temp2['distance_time'].min()) / ( temp2['distance_time'].max() - temp2['distance_time'].min())

  class_data_2 = pd.merge(class_data_1, temp2, left_index=True, right_index=True, how='right')

  return class_data_2




# 최종 추천 함수
def recommendation(user_input):

  classroom_org = make_distance(user_input)
  classroom_org = classroom_org.rename(columns={'building_x':'building'})
  classroom_org = classroom_org.drop(columns=["building_y"])
  classroom_0 = classroom_org.drop(columns=["building", "room"]) # 코사인 유사도 계산을 위해 불필요한 column 제거
  del user_input[0:2] # 코사인 유사도 계산을 위해 불필요한 column 제거

  # 정규화 - max min Normalization 사용 : (x-min(x) /(max(x)-min(x))
  total = classroom_0.append(classroom_0.iloc[-1], ignore_index=True)
  user_input.append(0) # 거리 정규화 목적
  classroom_0.iloc[-1] = user_input
  total = minmax_norm(total)
  classroom_0 = total.iloc[1:-1]
  user_input = total.tail(n=1)
  user_input = user_input.drop(columns=["distance_time"])
  classroom = classroom_0.drop(columns=["distance_time"]) # 코사인 유사도 계산을 위해 불필요한 column 제거


  # 사용자 Input - 강의실 Data 간의 코사인 유사도 계산 및 내림차순 정렬
  result = [] # 모든 강의실에 대한 코사인 유사도 값 저장 리스트

  for idx, row in classroom.iterrows():

    similarity_org = [idx, cos_sim(user_input.values, row.values)[0]]
    similarity_weight = similarity_org

    # 거리에 대한 가중치 부여
    similarity_weight[1] *= (1 - classroom_0.loc[idx]['distance_time']) 


    # 콘센트 필요도에 대한 가중치 부여
    similarity_weight[1] *= (1 + (1 - abs(user_input.loc[82]['plug_necessity'] - row['plug_necessity'])) * 0.86 )


    # 창문 유무에 대한 가중치 부여
    similarity_weight[1] *= (1 + (1 - abs(user_input.loc[82]['ac_bool'] - row['ac_bool'])) * 0.6 )


    # 책상 형태에 대한 가중치 부여
    similarity_weight[1] *= (1 + (1 - abs(user_input.loc[82]['table'] - row['table'])) * 0.68 )


    # 의자 형태에 대한 가중치 부여
    similarity_weight[1] *= (1 + (1 - abs(user_input.loc[82]['chair'] - row['chair'])) * 0.68 )

    result.append(similarity_weight)  # 결과값 리스트로 저장
  
  result_df = pd.DataFrame(result,columns= ['idx', 'CosineSimilarity_weight']) 
  result_df = result_df.sort_values(by='CosineSimilarity_weight' ,ascending=False) # 내림차순 정렬
  final_df =  pd.merge(classroom_org, result_df, left_index=True, right_index=True, how='right') #result_df = pd.merge(result_df, top18_org, on= 'idx', how ='left') 같은 방법
  final_df = final_df.iloc[1:10] # 상위 9개의 강의실 추출 
  final_df =  final_df[['building', 'room']].values
 
  return final_df





 # 예시 Input 
user_1 = [1, 'CJ법학관', 4, 1, 1, 1] 
print(recommendation(user_1))


  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
class_data_1
    building             room  plug_necessity  ac_bool  table  chair
0      CJ법학관              315               2        1      0      1
1      CJ법학관              316               4        1      0      1
2      CJ법학관              314               2        0      1      1
3        L-P  436(SUPEX HALL)               1        1      1      1
4        L-P              205               2        1      1      0
..       ...              ...             ...      ...    ...    ...
77     하나스퀘어             B120               1        1      0      1
78     하나스퀘어             B119               2        1      0      0
79  현대자동차경영관             B202               2        1      1      1
80  현대자동차경영관             B201               2        1      1      1
81  현대자동차경영관             B206               1        1      0      1

[82 rows x 6 columns]
     bu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
# 강의실 모델 초안 (사용X)


import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import numpy as np
import requests
import json
from haversine import haversine
from google.colab import drive
drive.mount('/content/drive')


# 강의실 데이터 전처리
class_data = pd.read_csv('/content/drive/MyDrive/데이터톤/data/강의실 생성2.csv', sep = ",",error_bad_lines=False) 
class_data.insert(3,'plugs_per_stu', class_data['plugs'] / class_data['max'])
class_data.insert(3, 'plug_necessity', pd.cut(class_data.plugs_per_stu, 5))
class_data['plug_necessity'] = class_data['plug_necessity'].astype('str')
class_data = class_data.replace(['(-0.001, 0.2]','(0.2, 0.4]', '(0.4, 0.6]', '(0.6, 0.8]','(0.8, 1.0]'],[1,2,3,4,5]) 
#class_data.insert(0, 'class_num', np.arange(1, 718).tolist()) 
class_data_1 = class_data.drop(columns=[ "max", "plugs_per_stu","plugs"]) # 호수 column 유지(코사인 유사도 계산 시에만 배제)
class_data_1= class_data_1.fillna(0)
print(class_data_1.head())


# 사용자 데이터 전처리
user_data = pd.read_csv('/content/drive/MyDrive/데이터톤/data/강의실 사용자 샘플 건물 위치.csv', sep = ",",error_bad_lines=False) 
user_data = user_data.drop(columns=["time"])
user_data.insert(3, 'ac_bool', 1) # 강의실 데이터에 맞춰 창문유무 column 추가
user_data = user_data.fillna(0)
print(user_data.head())


# 강의실 위도경도 데이터
building_lon_lat = pd.read_excel('/content/drive/MyDrive/데이터톤/data/강의실_위도경도 수정본.xlsx')
classroom = class_data_1
classroom.reset_index(inplace=True)
classroom.rename(columns = {'index' : 'idx'}, inplace = True)



# user input : 사용자 번호, 건물명, 콘센트 필요도, (창문 유무),책상형태(개인용=1,다인용=0), 의자형태(일체=1,분리=0)
# 거리 상위 18위(2배수) 이내의 강의실 추출 함수
def top18_classroom(user):

  # 가상 강의실에 위치정보 outer join 
  cat_loc = pd.merge(classroom, building_lon_lat, on= 'building', how ='left' )
  if cat_loc[['lat','lon']].isnull().values.any():
    print('Fail outer join room')
  else:
    print('Success outer join room')


  # 사용자 선택 건물 - 강의실 사이 거리구하기

  current_lat = user[-1]
  current_lon = user[-2]
  print(current_lat, current_lon)

  ls = []
  df_final = pd.DataFrame(columns=['building', 'distance'])

  for idx,row in building_lon_lat.iterrows():
    start = (round(float(current_lat), 6), round(float(current_lon), 6))
    goal = (round(float(row['lat']), 6),round(float(row['lon']), 6))
    ls.append((row[0],haversine(start,goal),round(float(row['lat']), 6),round(float(row['lon']), 6)))
    #print((idx,haversine(start,goal),row['lat'],row['lon']))
    ls = sorted(ls, key = lambda x: x[1], reverse = False)

  for row in ls:
    request_str = 'http://router.project-osrm.org/route/v1/foot/' + str(current_lon) + ',' +str(current_lat)+ ';' + str(row[3]) + ',' + str(row[2]) + '?overview=false'

    r = requests.get(request_str)
    routes = json.loads(r.content)
    distance = routes.get('routes')[0]['distance']
    df_final.loc[len(df_final)] = [row[0],distance]

    df_final.sort_values('distance', inplace=True)

  top_10 = df_final
  print('top_10')
  print(top_10)


  # 가까운 강의실 순서대로 강의실 나열
  # outer join 
  closer_classroom = pd.merge(top_10, cat_loc, on= 'building', how ='left' )
  if closer_classroom.isnull().values.any():
    closer_classroom = closer_classroom.dropna(axis=0)
    if closer_classroom.isnull().values.any():
      print('Fail outer join')
    else: 
      print('Success outer join')
  else:
    print('Success outer join')

  closer_classroom = closer_classroom.iloc[1:19]
  print("closer_classroom")
  print(closer_classroom.head())
  return closer_classroom



# 코사인유사도 계산 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))


# max min 정규화 함수
def minmax_norm(df_input):
    return df_input.apply(lambda x: (x-x.min())/ (x.max() - x.min()), axis=1)


# 최종 강의실 추천 함수
def recomm(input):
  # 사용자 input 값에 위치정보 outer join 
  for idx, row in building_lon_lat.iterrows():
    if row['building'] == input[1]:
      input.append(row['lon'])
      input.append(row['lat'])
      break


  # 코사인 유사도 계산을 위해 불필요한 column 제거
  top18_org = top18_classroom(input) 
  top18 = top18_org.drop(columns=["building", "distance", "idx", "room"]) # idx 주목
  del input[0:2]


  # 정규화 - max min Normalization 사용 : (x-min(x) /(max(x)-min(x))
  total = top18.append(top18.iloc[-1], ignore_index=True)
  top18.iloc[-1] = input
  total = minmax_norm(total)
  top18 = total.iloc[1:19].values
  input = total.tail(n=1).values
  #print(top18)
  #print(input)


  # (드디어) 코사인 유사도 계산 및 내림차순 정렬
  top18_len = len(top18)
  result = []
  index = top18_org['idx'].values
  for i in range(top18_len):
    result.append([index[i], cos_sim(input, top18[i])]) # 결과값이 리스트로 저장(i를 idx로 대체 필요)
  result_df = pd.DataFrame(result,columns= ['idx', 'CosineSimilarity']) 
  result_df = result_df.sort_values(by='CosineSimilarity' ,ascending=False) # 내림차순 정렬
  result_df = result_df.iloc[1:10]
  # merge(건물, 호수)
  result_df = pd.merge(result_df, top18_org, on= 'idx', how ='left' )
  result_df = result_df[['building', 'room']].values
 # print(result_df)
  return result_df


user1 = [1, 'CJ법학관', 4, 1, 1, 1] # 예시 input

recomm(user1) # 추천 모델



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  building  room  plug_necessity  ac_bool  table  chair
0    CJ법학관  512A               2        1      0      1
1    하나스퀘어  B119               1        1      1      0
2    하나스퀘어  B120               1        0      1      1
3    R&D센터   511               2        1      1      1
4    R&D센터   513               1        1      0      0
   user_num building  plug_necessity  ac_bool  desk_shape  chair_shape
0         1   우정간호학관               5        1           1            1
1         2    하나과학관               4        1           1            1
2         3    로봇융합관               4        1           1            0
3         4    이학관별관               5        1           1            0
4         5     미디어관               5        1           1            0
Success outer join room
37.5912182291962 127.033394748136
top_10
      building distance
0        CJ법학관      